In [2]:
!pip install groq python-dotenv numpy tqdm datasets

In [3]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

/home/woojin/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [9]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello! I'm excited to help you with any math problems you have. What kind of math are you working on? Do you have a specific problem you're stuck on or a concept you're trying to understand? Let me know and I'll do my best to assist you!


#### GSM8K 데이터셋 확인해보기

In [6]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [12]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [14]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

            if isinstance(predicted_answer, str):
                predicted_answer = float(predicted_answer.replace(",", ""))
            
            diff = abs(predicted_answer - correct_answer)
            is_correct = diff < 1e-5 if predicted_answer is not None else False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [10]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [1]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [15]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:02<00:02,  2.12it/s]

Progress: [5/10]
Current Acc.: [20.00%]


100%|██████████| 10/10 [00:04<00:00,  2.09it/s]

Progress: [10/10]
Current Acc.: [30.00%]


In [16]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!


#0-shot

PROMPT = construct_direct_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_0.txt")



#3-shot

PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_3.txt")



#5-shot

PROMPT = construct_direct_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_5.txt")



 10%|█         | 5/50 [00:02<00:19,  2.32it/s]

Progress: [5/50]
Current Acc.: [20.00%]


 20%|██        | 10/50 [00:05<00:25,  1.56it/s]

Progress: [10/50]
Current Acc.: [10.00%]


 30%|███       | 15/50 [00:08<00:18,  1.85it/s]

...

Progress: [40/50]
Current Acc.: [17.50%]


 90%|█████████ | 45/50 [02:16<00:18,  3.61s/it]

Progress: [45/50]
Current Acc.: [17.78%]


100%|██████████| 50/50 [02:25<00:00,  2.92s/it]

Progress: [50/50]
Current Acc.: [18.00%]


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [11]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "Instruction:\nSolve the following mathematical question step by step, and then generate the final answer after a tag, 'Answer:'.\n" #TODO: 프롬프트를 작성해주세요!
    

    for i in range(num_examples):
        idx = sampled_indices[i]
        cur_question = train_dataset['question'][idx]
        cur_answer = train_dataset['answer'][idx].strip()
        
        #TODO: CoT example을 만들어주세요!
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nLet's think step by step."

    return prompt

In [ ]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!


#0-shot

PROMPT = construct_CoT_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_0.txt")



#3-shot

PROMPT = construct_CoT_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_3.txt")



#5-shot

PROMPT = construct_CoT_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_5.txt")





 10%|█         | 5/50 [00:03<00:44,  1.02it/s]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [00:06<00:22,  1.78it/s]

Progress: [10/50]
Current Acc.: [60.00%]


 30%|███       | 15/50 [00:08<00:17,  2.00it/s]

...

Progress: [40/50]
Current Acc.: [70.00%]


 90%|█████████ | 45/50 [09:44<01:04, 12.91s/it]

Progress: [45/50]
Current Acc.: [71.11%]


100%|██████████| 50/50 [10:50<00:00, 13.01s/it]

Progress: [50/50]
Current Acc.: [70.00%]


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [12]:
### 자유롭게 수정해도 됩니다! 완전히 새로 함수를 만들어도 돼요.
#강의안에서 배웠던 CoT-SC를 구현했습니다

import re
import time
from collections import Counter
from tqdm import tqdm

# ✅ 모델 응답에서 정답 숫자만 추출 & 정규화
def extract_and_normalize_answer(response: str) -> str:
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)

    results = []
    for match in matches:
        if match.group(1):
            results.append(match.group(1).replace(",", ""))
        elif match.group(2):
            results.append(match.group(2).replace(",", ""))

    if not results:
        fallback_regex = r"\$?([0-9,]+)"
        fallback_matches = re.finditer(fallback_regex, response, re.MULTILINE)
        results.extend([m.group(1).replace(",", "") for m in fallback_matches])

    return results[-1] if results else None


def run_sc_benchmark_test(
    dataset,
    prompt_template: str,
    num_samples: int = 50,
    num_generations: int = 5,
    VERBOSE: bool = False
):
    results = []
    correct = 0

    for i in tqdm(range(num_samples)):
        example = dataset[i]
        question = example["question"]
        gt_answer_text = example["answer"]
        gt_answer = extract_and_normalize_answer(gt_answer_text)

        prompt = prompt_template.format(question=question)
        generations = []
        parsed_answers = []

        for _ in range(num_generations):
            # 자동 재시도
            for attempt in range(3):
                try:
                    response = client.chat.completions.create(
                        model="llama3-70b-8192",
                        messages=[{"role": "user", "content": prompt}],
                        temperature=1.0
                    )
                    output = response.choices[0].message.content.strip()
                    generations.append(output)
                    final_ans = extract_and_normalize_answer(output)
                    if final_ans is not None:
                        parsed_answers.append(final_ans)
                    break
                except Exception as e:
                    print(f"[Attempt {attempt+1}] Error: {e}")
                    time.sleep(1)

        if parsed_answers:
            predicted_answer, count = Counter(parsed_answers).most_common(1)[0]
        else:
            predicted_answer = "N/A"

        is_correct = (predicted_answer == gt_answer)

        if is_correct:
            correct += 1

        results.append({
            "question": question,
            "correct_answer": gt_answer,
            "predicted_answer": predicted_answer,
            "correct": is_correct,
            "raw_generations": generations,
            "parsed_answers": parsed_answers
        })

        if VERBOSE:
            print(f"\n[{i}] GT: {gt_answer} | Votes: {parsed_answers} | Majority: {predicted_answer}")

    accuracy = correct / num_samples * 100
    return results, accuracy








In [ ]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!


#0-shot


prompt_template = construct_CoT_prompt(0)  # 0CoT prompt
results, acc = run_sc_benchmark_test(
    dataset=gsm8k_test,
    prompt_template=prompt_template,
    num_samples=50,
    num_generations=5,  # self-consistency 횟수
    VERBOSE=True
)
save_final_result(results, acc, "my_prompting_0.txt")



  2%|▏         | 1/50 [00:05<04:24,  5.39s/it]


[0] GT: 18 | Votes: ['18', '18', '18', '18', '18'] | Majority: 18


  4%|▍         | 2/50 [00:15<06:21,  7.94s/it]


[1] GT: 3 | Votes: ['3', '3', '3', '3', '3'] | Majority: 3


  6%|▌         | 3/50 [00:31<09:08, 11.68s/it]

...


[25] GT: 2 | Votes: [] | Majority: N/A
[Attempt 1] Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01k1gtn3fcfkw95b53fvs24h2z` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499852, Requested 627. Please try again in 1m22.672199999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
[Attempt 2] Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01k1gtn3fcfkw95b53fvs24h2z` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499845, Requested 627. Please try again in 1m21.4372s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
[Attempt 3] Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in org

 54%|█████▍    | 27/50 [14:29<09:04, 23.66s/it]


[26] GT: 243 | Votes: [] | Majority: N/A
[Attempt 1] Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01k1gtn3fcfkw95b53fvs24h2z` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499745, Requested 621. Please try again in 1m3.0924s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
[Attempt 2] Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01k1gtn3fcfkw95b53fvs24h2z` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499738, Requested 621. Please try again in 1m1.8684s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
[Attempt 3] Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organiza

 54%|█████▍    | 27/50 [15:05<12:51, 33.55s/it]


KeyboardInterrupt: 

In [ ]:

#3-shot



prompt_template = construct_CoT_prompt(3)  # 0CoT prompt
results, acc = run_sc_benchmark_test(
    dataset=gsm8k_test,
    prompt_template=prompt_template,
    num_samples=50,
    num_generations=5,  # self-consistency 횟수
    VERBOSE=True
)
save_final_result(results, acc, "my_prompting_3.txt")



In [13]:


#5-shot


prompt_template = construct_CoT_prompt(5)  # CoT prompt
results, acc = run_sc_benchmark_test(
    dataset=gsm8k_test,
    prompt_template=prompt_template,
    num_samples=50,
    num_generations=5,  # self-consistency 횟수
    VERBOSE=True
)
save_final_result(results, acc, "my_prompting_5.txt")








  2%|▏         | 1/50 [00:07<06:23,  7.82s/it]


[0] GT: 18 | Votes: ['18', '18', '18', '18', '18'] | Majority: 18


  4%|▍         | 2/50 [01:15<34:24, 43.01s/it]


[1] GT: 3 | Votes: ['3', '3', '3', '3', '3'] | Majority: 3


  6%|▌         | 3/50 [02:26<43:37, 55.69s/it]

...


[47] GT: 800 | Votes: ['800', '800', '800', '800', '800'] | Majority: 800


 98%|█████████▊| 49/50 [57:39<01:11, 71.50s/it]


[48] GT: 8 | Votes: ['8', '8', '8', '8', '8'] | Majority: 8


100%|██████████| 50/50 [58:50<00:00, 70.61s/it]


[49] GT: 30 | Votes: ['30', '30', '30', '30', '30'] | Majority: 30


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!